<center><h1>Predicting Energy consumption - XGBoost and Decision tree<h1></center>

In [ ]:
import numpy as np
import pandas as pd

<div id="Understanding the dataset">
    <h2>Understanding the dataset</h2>
</div>


This Energy consumption dataset Contains information from https://www.kaggle.com/claytonmiller/annual-energy-consumption-from-singapore-buildings accessed on 07/08/2021 from https://data.gov.sg/dataset/building-energy-performance-data which is made available under the terms of the Singapore Open Data Licence version 1.0 https://data.gov.sg/open-data-licence
 


Imagine an Energy provider has recorded its customer buildings energy usage patterns, Classifying the Buildings annual energy usage as High or Low. If Energy use intensity data can be used to predict and support the estimated energyusage of new buildings.It is a Classification problem.  we need to build a model to be used to predict class of a new buildings.we use Decision tree to classify and use XGBoost classifier to improve accuracy.  

We use energyuseintensity-year1,energyuseintensity-year2, buildingsize,greenmarkstatus,grossfloorareausage patterns. 


In this notebook we build a classifier, to predict the class of newer buildings.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('../input/energy-consumption-data20182019/Data/Dataset2018.csv')
df2 = pd.read_csv('../input/energy-consumption-data20182019/Data/Dataset2019.csv')

frames = [df,df2]
my_data = pd.concat(frames)

In [ ]:
my_data['energyuseintensity(year1)'] = pd.to_numeric(my_data['energyuseintensity(year1)'],errors = 'coerce')
my_data['energyuseintensity(year2)'] = pd.to_numeric(my_data['energyuseintensity(year2)'],errors = 'coerce')

In [ ]:
newdata = pd.DataFrame(my_data)

<div id="data">
    <h2>Data Visualization </h2> 
</div>


In [ ]:

newdata.describe().T

In [ ]:
newdata.head()

In the dataset, energyuseintensity,grossfloorarea appear to be right skewed

In [ ]:
newdata.hist(column='grossfloorarea', bins=50)
newdata.hist(column='energyuseintensity(year1)', bins=50)
newdata.hist(column='energyuseintensity(year2)', bins=50)


<div id="data">
    <h2>Data Cleaning</h2> 
</div>

We analyse the  dataset for null rows, remove the null rows of the columns we aim to use for bulding our classifcation model

In [ ]:
newdata.isnull().sum()

In [ ]:
data = newdata.dropna(subset=['energyuseintensity(year1)','buildingsize','energyuseintensity(year2)','greenmarkstatus','grossfloorarea'])


In [ ]:
data.isnull().sum()

<h3>Categorical variable for target column:<h3>

In this part, we create a conditional column which gives us a categorical variable based on the EUI values of Buildings 

In [ ]:
row_indexes=data[data['energyuseintensity(year1)']>=200].index
data.loc[row_indexes,'EUI(Year1)']="High"
row_indexes=data[data['energyuseintensity(year1)']<200].index
data.loc[row_indexes,'EUI(Year1)']="Low"

row_indexes=data[data['energyuseintensity(year2)']>=200].index
data.loc[row_indexes,'EUI(Year2)']="High"
row_indexes=data[data['energyuseintensity(year2)']<200].index
data.loc[row_indexes,'EUI(Year2)']="Low"





In [ ]:
data.head()

In [ ]:
dataframe = pd.DataFrame(data)

We Pop and insert the columns that we are gonna use in the beginning for prediction purposes after fitting model.

In [ ]:
sixth_column = dataframe.pop('EUI(Year2)')
dataframe.insert(0, 'EUI(Year2)', sixth_column)


fifth_column = dataframe.pop('grossfloorarea')
dataframe.insert(0, 'grossfloorarea', fifth_column)


fourth_column = dataframe.pop('greenmarkstatus')
dataframe.insert(0, 'greenmarkstatus', fourth_column)

third_column = dataframe.pop('energyuseintensity(year2)')
dataframe.insert(0, 'energyuseintensity(year2)', third_column)

second_column = dataframe.pop('energyuseintensity(year1)')
dataframe.insert(0, 'energyuseintensity(year1)', second_column)

first_column = dataframe.pop('buildingsize')
dataframe.insert(0, 'buildingsize', first_column)


Checking the data for null values

In [ ]:
dataframe.isnull().sum()

Note that there are type float64 objects, Decision tree classifier takes float32 as input thus we need to convert it.

In [ ]:
print (dataframe.dtypes)

In [ ]:
my_dataframe = dataframe.astype({c: np.float32 for c in dataframe.select_dtypes(include='float64').columns})

In [ ]:
print (my_dataframe.dtypes)

In [ ]:
my_dataframe.head()

<h3>Pre-Processing<h3>

X is the Data present and
Y is the Target

In [ ]:
X = my_dataframe[['buildingsize','energyuseintensity(year1)','energyuseintensity(year2)','greenmarkstatus','grossfloorarea']].values
X[0:4]

We Encode categorical values as decision tree only takes values as inputs

In [ ]:
le = LabelEncoder()
le.fit(['Small','Large']) 
X[:,0] = le.transform(X[:,0])


le_ = LabelEncoder()
le_.fit([ 'No','Yes'])
X[:,3] = le_.transform(X[:,3])





X[0:6]


In [ ]:
y = my_dataframe['EUI(Year2)']

y[0:10]

Verifying shape of the data to be same

In [ ]:
print(X.shape,y.shape)

<h2>Splitting data and modelling<h2>
    

We use train_test_split to split the data for training and validation

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

In [ ]:
DecisionTree = DecisionTreeClassifier(criterion="entropy", max_depth = 5)


In [ ]:
DecisionTree.fit(X_trainset,y_trainset)

<h2>Prediciton and accuracy<h2>

In [ ]:
predTree = DecisionTree.predict(X_testset)

we can see that our model predicts most of the Predicitions right

In [ ]:
print (predTree [0:10])
print (y_testset [0:10])


Accuracy of the model is found to be 82% and we aim to improve it by using XGBoost

In [ ]:
from sklearn import metrics
import matplotlib.pyplot as plt
print("DecisionTrees's Accuracy: ",metrics.accuracy_score(y_testset, predTree))


<h2>Visualizing the Decision tree<h2>

Installing required libraries to visualize decision tree

In [ ]:
!conda install -c conda-forge pydotplus -y
!conda install -c conda-forge python-graphviz -y

In [ ]:
from  io import StringIO
import graphviz
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from sklearn import tree
%matplotlib inline 

In [ ]:
from sklearn.tree import export_graphviz
featureNames = dataframe.columns[0:5]
targetNames = dataframe["EUI(Year2)"].unique().tolist()
export_graphviz(DecisionTree, out_file='tree_limited.dot',feature_names=featureNames,
                class_names = np.unique(y_trainset),
                )
!dot -Tpng tree_limited.dot -o tree_limited.png -Gdpi=600



In [ ]:
from IPython.display import Image
Image(filename = 'tree_limited.png')

The above decision tree shows the classification of EUI for newer buildings

<h1>New model:XGBoost Classifier<h1>

Label encoding of the target vector is done as XGBoost requires the target to be values

In [ ]:
le_y = LabelEncoder()
le_y.fit(['Low','High'])
y= le_y.transform(y)

In [ ]:
from xgboost import XGBClassifier
my_model = XGBClassifier()

X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.2, random_state=3)



We create a model with random hyperparameters that are later Tuned to achieve maximum accuracy

In [ ]:
my_model = XGBClassifier(n_estimators=1000, learning_rate=0.01,max_depth=3,min_child_weight=2,                      
 subsample=0.8,
 nthread=4,
 colsample_bytree = 0.8,                       
 scale_pos_weight=0.1,
 seed=100
 )
my_model.fit(X_trainset, y_trainset, early_stopping_rounds=5, 
             eval_set=[( X_testset, y_testset)], verbose=False)

Accuracy of the model is found to be 84% using XGBoost classifier!

In [ ]:
from sklearn.metrics import accuracy_score

y_prediction = my_model.predict(X_testset)
predictions = [round(value) for value in y_prediction]
# evaluate predictions
accuracy = accuracy_score(y_testset, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))